In [ ]:
def recommend_similar_items(recommender, item_profiles, target_item_id, ending_numbers, top_n=10):
    """특정 상품의 ID를 입력받아 유사한 상품을 카테고리와 설명 유사도 기준으로 추천합니다."""
    recommendations = []
    
    # 대상 상품 정보 가져오기
    target_item = item_profiles.get(target_item_id)
    if not target_item:
        print("Invalid item ID. Please check and try again.")
        return []

    target_item_name = target_item.get('name')
    target_item_categories = target_item.get('categories')
    target_item_embedding = target_item.get('embedding', None)

    if not target_item_categories or not target_item_name:
        print("The target item lacks necessary information.")
        return []

     # 타겟 상품의 이름과 카테고리 출력
    target_category_path = get_category_path_string(target_item_categories)
    print(f"\n타겟 상품: {target_item_name}, 카테고리: {target_category_path}")

    # 모든 다른 상품에 대해 유사도 계산
    for item_id, item_data in item_profiles.items():
        if item_id == target_item_id:  # 자신은 제외
            continue

        item_name = item_data.get('name')
        item_categories = item_data.get('categories')
        item_embedding = item_data.get('embedding', None)

        if not item_name or not item_categories:
            continue

        # 상품 이름이 입력된 숫자로 끝나는지 확인
        if not any(item_name.endswith(str(num)) for num in ending_numbers):
            continue

        # 카테고리 유사도 계산
        max_category_similarity = 0
        if all(node in recommender.graph for node in target_item_categories) and \
           all(node in recommender.graph for node in item_categories):
            category_similarity = recommender.wu_palmer_similarity(target_item_categories, item_categories)
            max_category_similarity = max(max_category_similarity, category_similarity)

        # 설명 유사도 계산
        max_description_similarity = 0
        if target_item_embedding and item_embedding:
            description_similarity = recommender.compute_description_similarity(target_item_embedding, item_embedding)
            max_description_similarity = max(max_description_similarity, description_similarity)

        # 추천 리스트에 추가
        if max_category_similarity > 0:
            recommendations.append({
                'item_id': item_id,
                'name': item_name,
                'category_similarity': max_category_similarity,
                'description_similarity': max_description_similarity
            })

    # 카테고리 유사도와 설명 유사도를 기준으로 정렬
    recommendations.sort(key=lambda x: (x['category_similarity'], x['description_similarity']), reverse=True)

    # 상위 n개의 추천 결과 반환
    return recommendations[:top_n]

# 테스트 코드
def main():
    # JSON 파일 불러오기
    json_path = 'item_profiles_with_embeddings.json'
    item_profiles = load_item_profiles(json_path)
    
    # 추천 시스템 초기화
    recommender = ProductRecommendationSystem()
    recommender.create_category_graph("left_joined_file.csv")

    # 사용자 입력으로 상품 ID를 받아 유사한 상품 추천
    target_item_id = input("Enter the item ID to find similar products: ")
    
    # 숫자 입력 받기
    ending_numbers = input("Enter numbers (1-5) to filter by (comma-separated): ")
    ending_numbers = [int(num.strip()) for num in ending_numbers.split(',') if num.strip().isdigit()]

    recommendations = recommend_similar_items(recommender, item_profiles, target_item_id, ending_numbers)

    if recommendations:
        print(f"\nTop {len(recommendations)} similar products to item ID {target_item_id}:")
        for rec in recommendations:
            item_id = rec['item_id']
            item_name = rec['name']
            category_similarity = rec['category_similarity']
            description_similarity = rec['description_similarity']
            
            category_path = item_profiles[item_id]['categories']
            print(f"\n상품 이름: {item_name}")
            print(f"카테고리: {get_category_path_string(category_path)}")
            print(f"카테고리 유사도: {category_similarity:.4f}")
            print(f"설명 유사도: {description_similarity:.4f}")
            print('-' * 20)

if __name__ == "__main__":
    main()